## Evaluation
The below code uses the [kudkudak/word-embeddings-benchmarks package][1] to evaluate the [`glove-wiki-gigaword-50`][3] embeddings on the [Google Analogy test set][2].

 [1]: https://github.com/kudkudak/word-embeddings-benchmarks (Package for evaluating word embeddings)
 [2]: https://aclweb.org/aclwiki/Google_analogy_test_set_(State_of_the_art) (Google analogy test set)
 [3]: https://github.com/RaRe-Technologies/gensim-data/releases/tag/glove-wiki-gigaword-50

In [1]:
import gensim.downloader as api

model = api.load('glove-wiki-gigaword-50')

In [2]:
reverse_dictionary = {
    vocab.index: term.encode('utf8')
    for term, vocab in model.vocab.items()
}

In [3]:
from annoy import AnnoyIndex

n_trees = 100
search_k = 100

annoy_index = AnnoyIndex(model.vector_size, metric='angular')
for vocab in model.vocab.values():
    annoy_index.add_item(vocab.index, model.vectors[vocab.index])
annoy_index.build(n_trees)

True

In [4]:
from web.datasets.analogy import fetch_google_analogy

dataset = fetch_google_analogy()

In [5]:
from __future__ import division

previous_category = dataset.category[0]
trials = {'semantic': [], 'syntactic': [], 'total': []}
missing_words = set()
skipped_trials = 0

def print_acc():
    print('- Semantic accuracy:\t{} %\n- Syntactic accuracy:\t{} %\n- Total accuracy:\t{} %'.format(
        float('nan') if not trials['semantic'] else sum(trials['semantic']) / len(trials['semantic']) * 100,
        float('nan') if not trials['syntactic'] else sum(trials['syntactic']) / len(trials['syntactic']) * 100,
        float('nan') if not trials['total'] else sum(trials['total']) / len(trials['total']) * 100,
    ))
    print('- Missing words:\t{} (skipped {} % of all trials)'.format(
        ', '.join(missing_words),
        float('nan') if not trials['total'] else skipped_trials / len(trials['total']) * 100,
    ))
    print('')

for X, y, category, category_high_level in zip(
            dataset.X,
            dataset.y,
            dataset.category,
            dataset.category_high_level
        ):

    if previous_category != category:
        previous_category = category
        print('Current category: {}'.format(category))
        print_acc()

    if X[0] in model.vocab and X[1] in model.vocab and X[2] in model.vocab and y in model.vocab:
        u1 = model.vectors[model.vocab[X[0]].index]
        v1 = model.vectors[model.vocab[X[1]].index]
        u2 = model.vectors[model.vocab[X[2]].index]
        v2_index = annoy_index.get_nns_by_vector(u2 + (v1 - u1), 1, search_k)[0]
        v2_word = reverse_dictionary[v2_index]
        print('{} - {} + {} = {} (correct answer: {})'.format(X[1], X[0], X[2], v2_word, y))
        trial_success = v2_word == y
    else:
        if X[0] not in dictionary:
            missing_words.add(X[0])
        if X[1] not in dictionary:
            missing_words.add(X[1])
        if X[2] not in dictionary:
            missing_words.add(X[2])
        if y not in dictionary:
            missing_words.add(y)
        skipped_trials += 1
        trial_success = False
    trials[category_high_level].append(trial_success)
    trials['total'].append(trial_success)

print('Finished!')
print_acc()

greece - athens + baghdad = iraq (correct answer: iraq)
greece - athens + bangkok = thailand (correct answer: thailand)
greece - athens + beijing = china (correct answer: china)
greece - athens + berlin = germany (correct answer: germany)
greece - athens + bern = bern (correct answer: switzerland)
greece - athens + cairo = egypt (correct answer: egypt)
greece - athens + canberra = greece (correct answer: australia)
greece - athens + hanoi = laos (correct answer: vietnam)
greece - athens + havana = cuba (correct answer: cuba)
greece - athens + helsinki = helsinki (correct answer: finland)
greece - athens + islamabad = islamabad (correct answer: pakistan)
greece - athens + kabul = afghanistan (correct answer: afghanistan)
greece - athens + london = london (correct answer: england)
greece - athens + madrid = madrid (correct answer: spain)
greece - athens + moscow = russia (correct answer: russia)
greece - athens + oslo = norway (correct answer: norway)
greece - athens + ottawa = ottawa (c

/home/witiko/.virtualenvs/word2bits-in-tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:39: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


germany - berlin + jakarta = thailand (correct answer: indonesia)
germany - berlin + kabul = afghanistan (correct answer: afghanistan)
germany - berlin + kampala = uganda (correct answer: uganda)
germany - berlin + kathmandu = nepal (correct answer: nepal)
germany - berlin + khartoum = sudan (correct answer: sudan)
germany - berlin + kiev = ukraine (correct answer: ukraine)
germany - berlin + kigali = rwanda (correct answer: rwanda)
germany - berlin + kingston = australia (correct answer: jamaica)
germany - berlin + libreville = gabon (correct answer: gabon)
germany - berlin + lilongwe = lilongwe (correct answer: malawi)
germany - berlin + lima = brazil (correct answer: peru)
germany - berlin + lisbon = portugal (correct answer: portugal)
switzerland - bern + bishkek = kyrgyzstan (correct answer: kyrgyzstan)
switzerland - bern + bratislava = slovakia (correct answer: slovakia)
switzerland - bern + brussels = france (correct answer: belgium)
switzerland - bern + bucharest = spain (corre

ukraine - kiev + rome = italy (correct answer: italy)
ukraine - kiev + roseau = jigawa (correct answer: dominica)
ukraine - kiev + santiago = costa (correct answer: chile)
ukraine - kiev + skopje = macedonia (correct answer: macedonia)
rwanda - kigali + kingston = wales (correct answer: jamaica)
rwanda - kigali + libreville = gabon (correct answer: gabon)
rwanda - kigali + lilongwe = lilongwe (correct answer: malawi)
rwanda - kigali + lima = peru (correct answer: peru)
rwanda - kigali + lisbon = portugal (correct answer: portugal)
rwanda - kigali + ljubljana = ljubljana (correct answer: slovenia)
rwanda - kigali + london = london (correct answer: england)
rwanda - kigali + luanda = angola (correct answer: angola)
rwanda - kigali + lusaka = lusaka (correct answer: zambia)
rwanda - kigali + madrid = madrid (correct answer: spain)
rwanda - kigali + managua = nicaragua (correct answer: nicaragua)
rwanda - kigali + manama = bahrain (correct answer: bahrain)
rwanda - kigali + manila = philip

italy - rome + tripoli = libya (correct answer: libya)
italy - rome + tunis = mali (correct answer: tunisia)
italy - rome + vaduz = vaduz (correct answer: liechtenstein)
italy - rome + valletta = ajaccio (correct answer: malta)
italy - rome + vienna = belgium (correct answer: austria)
italy - rome + vientiane = laos (correct answer: laos)
italy - rome + vilnius = lithuania (correct answer: lithuania)
italy - rome + warsaw = poland (correct answer: poland)
italy - rome + windhoek = windhoek (correct answer: namibia)
italy - rome + yerevan = moldova (correct answer: armenia)
italy - rome + zagreb = croatia (correct answer: croatia)
italy - rome + abuja = nigeria (correct answer: nigeria)
italy - rome + accra = ghana (correct answer: ghana)
italy - rome + algiers = algiers (correct answer: algeria)
italy - rome + amman = thailand (correct answer: jordan)
italy - rome + ankara = turkey (correct answer: turkey)
italy - rome + antananarivo = togo (correct answer: madagascar)
italy - rome + a

...

walking - walk + say = say (correct answer: saying)
walking - walk + scream = scream (correct answer: screaming)
walking - walk + see = see (correct answer: seeing)
walking - walk + shuffle = shuffle (correct answer: shuffling)
walking - walk + sing = sing (correct answer: singing)
walking - walk + sit = sitting (correct answer: sitting)
walking - walk + slow = slow (correct answer: slowing)
walking - walk + swim = swim (correct answer: swimming)
walking - walk + think = think (correct answer: thinking)
walking - walk + vanish = vanish (correct answer: vanishing)
writing - write + code = code (correct answer: coding)
writing - write + dance = dance (correct answer: dancing)
writing - write + debug = debugging (correct answer: debugging)
writing - write + decrease = decrease (correct answer: decreasing)
writing - write + describe = describe (correct answer: describing)
writing - write + discover = discover (correct answer: discovering)
writing - write + enhance = enhance (correct answer

polish - poland + japan = japanese (correct answer: japanese)
polish - poland + korea = korean (correct answer: korean)
polish - poland + macedonia = macedonian (correct answer: macedonian)
polish - poland + malta = latvian (correct answer: maltese)
polish - poland + mexico = mexican (correct answer: mexican)
polish - poland + moldova = ukrainian (correct answer: moldovan)
polish - poland + netherlands = dutch (correct answer: dutch)
polish - poland + norway = norwegian (correct answer: norwegian)
portuguese - portugal + russia = russian (correct answer: russian)
portuguese - portugal + slovakia = slovak (correct answer: slovakian)
portuguese - portugal + spain = spanish (correct answer: spanish)
portuguese - portugal + sweden = swedish (correct answer: swedish)
portuguese - portugal + switzerland = swiss (correct answer: swiss)
portuguese - portugal + thailand = thai (correct answer: thai)
portuguese - portugal + ukraine = ukrainian (correct answer: ukrainian)
portuguese - portugal + 

screamed - screaming + thinking = thinking (correct answer: thought)
screamed - screaming + vanishing = meaninglessness (correct answer: vanished)
screamed - screaming + walking = walking (correct answer: walked)
screamed - screaming + writing = writing (correct answer: wrote)
screamed - screaming + dancing = dancing (correct answer: danced)
screamed - screaming + decreasing = decreasing (correct answer: decreased)
screamed - screaming + describing = comments (correct answer: described)
screamed - screaming + enhancing = enhancing (correct answer: enhanced)
screamed - screaming + falling = falling (correct answer: fell)
screamed - screaming + feeding = feeding (correct answer: fed)
screamed - screaming + flying = flying (correct answer: flew)
screamed - screaming + generating = generating (correct answer: generated)
screamed - screaming + going = going (correct answer: went)
screamed - screaming + hiding = hiding (correct answer: hid)
screamed - screaming + hitting = hitting (correct a

hands - hand + bird = bird (correct answer: birds)
hands - hand + bottle = bottle (correct answer: bottles)
hands - hand + building = building (correct answer: buildings)
hands - hand + car = car (correct answer: cars)
hands - hand + cat = cat (correct answer: cats)
hands - hand + child = child (correct answer: children)
hands - hand + cloud = cloud (correct answer: clouds)
hands - hand + color = color (correct answer: colors)
hands - hand + computer = computer (correct answer: computers)
hands - hand + cow = cow (correct answer: cows)
hands - hand + dog = dog (correct answer: dogs)
hands - hand + dollar = dollar (correct answer: dollars)
hands - hand + donkey = donkey (correct answer: donkeys)
hands - hand + dream = dream (correct answer: dreams)
hands - hand + eagle = eagle (correct answer: eagles)
hands - hand + elephant = elephant (correct answer: elephants)
hands - hand + eye = eye (correct answer: eyes)
hands - hand + finger = finger (correct answer: fingers)
hands - hand + goat 

swims - swim + work = focused (correct answer: works)
swims - swim + write = translating (correct answer: writes)
swims - swim + decrease = decreases (correct answer: decreases)
swims - swim + describe = anecdote (correct answer: describes)
swims - swim + eat = ate (correct answer: eats)
swims - swim + enhance = enhance (correct answer: enhances)
swims - swim + estimate = estimate (correct answer: estimates)
swims - swim + find = find (correct answer: finds)
swims - swim + generate = generates (correct answer: generates)
swims - swim + go = going (correct answer: goes)
swims - swim + implement = implementing (correct answer: implements)
swims - swim + increase = increase (correct answer: increases)
swims - swim + listen = listens (correct answer: listens)
swims - swim + play = play (correct answer: plays)
swims - swim + predict = predict (correct answer: predicts)
swims - swim + provide = providing (correct answer: provides)
swims - swim + say = believe (correct answer: says)
swims - s